In [1]:
source('../common.r')

In [ ]:
d <- data.rawmix(where="name like 'v0.28.1%' and nclients = 4 and duration = 30 and length = 4 and rate = 100")
d <- subset(d, commute_ratio == 0.5 & alpha == 0.6)

In [ ]:
sql("SELECT cc_ph, avg(throughput) FROM d WHERE nthreads = 96 GROUP BY cc_ph")

# Best Zipf

In [2]:
d <- data.rawmix(where="name like 'v0.28.1%' and nclients = 4 and duration = 30 and length = 4 and rate = 100 and timeout_scaling = 1000 and phase_limit = 100")
d$zipf <- num(d$alpha)
d <- subset(d, commute_ratio == 0.5)

In [ ]:
unique(d$ccmode)

In [3]:
d.mean <- ddply(d, .(nthreads,cc_ph,zipf,phasing,cc), summarize, throughput=mean(throughput))
d.max <- ddply(d.mean, .(cc_ph,zipf), summarize, throughput=max(throughput))

In [4]:
{
  d.better <- subset(d.max, cc_ph == BETT+COMB+PH)
  d.notxn  <- subset(d.max, cc_ph == NOTXN)
  zipf.speedups <- d.better$throughput / d.notxn$throughput
}
mean(zipf.speedups)

[1] 0.8248753

In [5]:
q <- sql("SELECT zipf, cc_ph, throughput FROM `d.max` WHERE zipf = 1.4 ORDER BY zipf")
q

,zipf,cc_ph,throughput
1,1.4,without transactions,10924.37
2,1.4,better boosting + combining + phasing,13555.6
3,1.4,better boosting + phasing,6724.25
4,1.4,boosting + combining + phasing,12459.71
5,1.4,boosting + phasing,6542.957
6,1.4,r/w locks + phasing,753.7955
7,1.4,boosting + combining,449.0317
8,1.4,boosting,1517.632
9,1.4,better boosting + combining,12841.4
10,1.4,better boosting,6668.132


## Speedups

In [6]:
speedups <- function(q) {
    speedup.base <- q[q$cc_ph == BETT+COMB+PH,]$throughput / q[q$cc_ph == RW+BASE,]$throughput
    speedup.nontxn <- q[q$cc_ph == BETT+COMB+PH,]$throughput / q[q$cc_ph == NOTXN,]$throughput
    cat("speedup over baseline:          " + round(speedup.base,2) + "\n")
    cat("speedup over non-transactional:  " + round(speedup.nontxn,2) + "\n")
}

In [7]:
speedups(q)

speedup over baseline:          48.84
speedup over non-transactional:  1.24


## Rawmix: Commute Ratio

In [8]:
d <- data.rawmix(where="name like 'v0.28.1%' and nclients = 4 and duration = 30 and length = 4 and rate = 100")
levels(d$phasing)

[1] "off" "on"

In [9]:
{
  d <- data.rawmix(where="name like 'v0.28.1%' and nclients = 4 and duration = 30 and length = 4 and rate = 100")
  d$zipf <- num(d$alpha)
  d.mix <- subset(d, nkeys == 1000 & zipf == 0.6)
  d <- subset(d.mix, select = c('rate', 'nthreads', 'cc_ph', 'cc', 'zipf', 'commute_ratio', 'phasing', 'timeout_scaling', 'throughput', 'op_timeouts'))
  d.mean <- ddply(d, .(nthreads,cc_ph,commute_ratio,phasing,cc), summarize, throughput=mean(throughput))
  d.max <- ddply(d.mean, .(cc_ph,phasing,cc,commute_ratio), summarize, throughput=max(throughput))
  d.mix <<- d.max
}

In [10]:
d.better <- subset(d.mix, cc_ph == BETT+COMB+PH)
d.notxn  <- subset(d.mix, cc_ph == NOTXN)

speedups <- d.better$throughput / d.notxn$throughput
speedups

[1] 0.7833409 0.7093968 0.6708936 0.6541169 0.6351011 0.6280014 0.6223550
 [8] 0.7610298 0.7110524 0.6718596 0.6522800 0.6449613 0.6230186 0.6297064

In [11]:
mean(speedups)

[1] 0.6712224

# Rubis

In [27]:
source('../common.r')

In [28]:
{
    #d <- data.rubis(where="duration = 60 and name like 'v0.29%' and nthreads <= 96")
    d <- db.csv(file = 'rubis-tput-vs-lat.csv')
    
    d$workload <- revalue(d$mix, c(
      'mixed'     = 'read-heavy',
      'bid-heavy' = 'bid-heavy'
    ))

    d.rubis <<- subset(d, lambda == 20 & grepl('read-heavy|bid-heavy', workload))
}

In [52]:
{
    d.mean <- ddply(d.rubis, .(nthreads,cc_ph,workload,phasing,cc), summarize, throughput=mean(throughput))
  d.max <- ddply(d.mean, .(cc_ph,phasing,cc,workload), summarize, throughput=max(throughput))
  q.rubis <<- d.max
}

In [53]:
q.rubis

,cc_ph,phasing,cc,workload,throughput
1,boosting + combining + phasing,on,boosting + combining,bid-heavy,14535.27
2,boosting + combining + phasing,on,boosting + combining,read-heavy,15065.83
3,boosting + phasing,on,boosting,bid-heavy,13956.6
4,boosting + phasing,on,boosting,read-heavy,15023.43
5,r/w locks + phasing,on,r/w locks,bid-heavy,8373.104
6,r/w locks + phasing,on,r/w locks,read-heavy,14390.65
7,boosting + combining,off,boosting + combining,bid-heavy,14444.97
8,boosting + combining,off,boosting + combining,read-heavy,14881.67
9,boosting,off,boosting,bid-heavy,13434.1
10,boosting,off,boosting,read-heavy,14710.85


In [54]:
{ q <- subset(q.rubis, grepl('bid-heavy', workload))
  cat("comb: " + q[q$cc_ph == COMB+PH,]$throughput + "\n")
  cat("base: " + q[q$cc_ph == RW+BASE,]$throughput + "\n")
}

comb: 14535.2711379783
base: 3375.6510676982


In [55]:
speedup <- function(q) {
  speedup.base <- q[q$cc_ph == COMB+PH,]$throughput / q[q$cc_ph == RW+BASE,]$throughput
  speedup.nontxn <- q[q$cc_ph == COMB+PH,]$throughput / q[q$cc_ph == NOTXN,]$throughput
  cat("  speedup over baseline:           " + round(speedup.base,2) + "\n")
  cat("  speedup over non-transactional:  " + round(speedup.nontxn,2) + "\n")
}

cat("bid-heavy  -> \n")
speedup(subset(q.rubis, grepl('bid-heavy', workload)))

cat("read-heavy -> \n")
speedup(subset(q.rubis, grepl('read-heavy', workload)))

bid-heavy  -> 
  speedup over baseline:           4.31
  speedup over non-transactional:  0.68
read-heavy -> 
  speedup over baseline:           1.44
  speedup over non-transactional:  0.75


# Retwis

In [57]:
d.retwis <- data.retwis(where="(name like '%v0.30%' and nclients = 4)")
d.retwis <- subset(d.retwis, async == 0 & txn_failed < 200 & total_time > 60 & total_time < 65 & (phasing == 'on' | disable_txns == 1))

The following `from` values were not present in `x`: better#0, better#1
The following `from` values were not present in `x`: better#0, better#1
The following `from` values were not present in `x`: better#0#off#0, better#1#off#0, better#0#on#0, better#1#on#0
The following `from` values were not present in `x`: simple:1
The following `from` values were not present in `x`: update_heavy


In [ ]:
unique(d.retwis$workload)

In [58]:
{
  d.mean <- ddply(d.retwis, .(nthreads,cc_ph,workload,phasing,cc), summarize, throughput=mean(throughput))
  d.max <- ddply(d.mean, .(cc_ph,phasing,cc,workload), summarize, throughput=max(throughput))
  q.retwis <<- d.max
}

In [ ]:
q.retwis <- sql("SELECT workload, cc_ph, avg(throughput) as throughput FROM `d.retwis` WHERE nthreads = 96 GROUP BY workload, cc_ph ORDER BY workload")
q.retwis

In [59]:
speedup <- function(q) {
  r.best <- max(subset(q, grepl('boosting', cc_ph))$throughput)
  r.notxn <- subset(q, cc_ph == NOTXN)$throughput
  r.base <- subset(q, cc_ph == RW+PH)$throughput
  cat("  speedup over baseline: " + round(r.best / r.base, 2) + "\n")
  cat("  speedup over notxn:    " + round(r.best / r.notxn, 2) + "\n")
}

cat("post-heavy -> \n")
speedup(subset(q.retwis, grepl('post-heavy', workload)))

cat("read-heavy -> \n")
speedup(subset(q.retwis, grepl('read-heavy', workload)))

post-heavy -> 
  speedup over baseline: 3.57
  speedup over notxn:    0.82
read-heavy -> 
  speedup over baseline: 0.96
  speedup over notxn:    0.72
